# Installation

In [1]:
!pip install gensim
!pip install PySastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 2.1 MB/s eta 0:00:00


# Packages

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import gensim
import nltk
from collections import OrderedDict
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.metrics.pairwise import cosine_distances
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Dataset

In [3]:
# Dictionary containing file paths for each dataset
file_paths = {
    "bandung": "/content/bandung.csv",
    "jakarta": "/content/jakarta.csv"
}

# Loop iterates over each key and path in file_paths
datasets = {key: pd.read_csv(path) for key, path in file_paths.items()}

# Data Preprocessing

In [4]:
class TextPreprocessing():
  def __init__(self, data, city):
    """
    Initialize TextPreprocessing class.

    Parameters:
    - data (dict): Dictionary containing dataset for different cities.
    - city (str): The city to be processed within the data.
    """
    self.data = data
    self.city = city
    self.column = 'metadata'

  def stemming(self):
    """
    Preprocess the corpus with stemming.

    Returns:
    - list: A list of stemmed documents.
    """
    stemmed_documents = []
    # Define the corpus
    corpus = self.data[self.city][self.column]
    # Preprocessing with stopword before stemming
    stopword_factory = StopWordRemoverFactory()
    remover = stopword_factory.create_stop_word_remover()
    # Stemming the stopword
    stemming_factory = StemmerFactory()
    stemmer = stemming_factory.create_stemmer()
    # append the cleaned text with stopword and stemming
    for doc in corpus:
      stemmed_doc = [stemmer.stem(word) for word in doc.split()]
      cleaned_text = remover.remove(' '.join(stemmed_doc))
      stemmed_documents.append(cleaned_text)
    return stemmed_documents

  def tokenizer(self, stemmed_documents):
    """
    Preprocess the stemmed corpus with a tokenizer.

    Parameters:
    - stemmed_documents (list): A list of preprocessed documents after stemming.

    Returns:
    - dict: Updated datasets containing tokenized strings.
    - list: A list of tokenized strings.
    """
    num_words = None
    oov_tok = "<OOV>"
    lower=True
    char_level = False
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    # Define the tokenizer
    tokenizer = Tokenizer(num_words=num_words,
                          filters=filters,
                          oov_token=oov_tok,
                          lower=lower,
                          char_level=char_level)
    # Fit tokenizer on stemmed documents
    tokenizer.fit_on_texts(stemmed_documents)
    # Tokenized texts to sequences
    tokenized_texts = tokenizer.texts_to_sequences(stemmed_documents)
    # Tokenized sequences to texts
    self.tokenized_strings = tokenizer.sequences_to_texts(tokenized_texts)
    # Create column 'metadata_tokenized' on the dataset
    self.data[self.city]['metadata_tokenized'] = self.tokenized_strings
    return self.data, self.tokenized_strings

# User Input

In [5]:
class User():
  def __init__(self, data):
    """
    Initialize User class with attributes for user inputs.
    """
    self.data = data
    self.city = None
    self.budget = None
    self.duration = None
    self.user_preferences = None
    self.combined_list = None

  # Define input function from user
  def input(self, city, budget, duration, user_preferences_1, user_preferences_2=None):
    """
    Define input function to capture user preferences and inputs.

    Parameters:
    - city (str): The city chosen by the user.
    - budget (float): The budget set by the user.
    - duration (int): The duration of the user's visit.
    - user_preferences_1 (str): The primary user preference.
    - user_preferences_2 (str, optional): Additional user preference (if any).

    Returns:
    - list: A combined list of tokenized metadata for the chosen city.
    - list: A list of user preferences.
    - int: The duration of the visit.
    - float: The budget for the trip.
    """
    self.city = str(city)
    self.budget = float(budget)
    self.duration = int(duration)
    self.user_preferences = [user_preferences_1]
    # If user_preferences is not None, then append user_preferences_2 to user_preferences
    if user_preferences_2 is not None:
      self.user_preferences.append(user_preferences_2)
    # Combine the column of 'metadata_tokenized' to one list
    self.combined_list = self.data[city]['metadata_tokenized'].tolist()
    return self.combined_list, self.user_preferences, self.duration, self.budget

# Models

## Word2vec

In [6]:
class GenerateNewPreference():
  def __init__(self):
    """
    Initialize GenerateNewPreferences class with default parameters.
    """
    self.user_preferences = None
    self.dataset = None
    self.vector_size = 100
    self.window = 500
    self.min_count = 2
    self.workers = 100

  def fit(self, dataset, user_preferences):
    """
    Fit method to generate new user preferences using Word2Vec.
    Parameters:
    - dataset (list): List of tokenized sentences or text data.
    - user_preferences (list): List of user preferences.
    Returns:
    - str: New user preference generated from Word2Vec model.
    """
    self.user_preferences = user_preferences
    self.dataset = dataset

    # If user preferences is not 1
    if len(self.user_preferences) != 1:
      # Tokenize the dataset
      tokenized_data = [gensim.utils.simple_preprocess(sentence) for sentence in dataset]
      # Model Word2Vec
      model = gensim.models.Word2Vec(tokenized_data,
                                     vector_size=self.vector_size,
                                     window=self.window,
                                     min_count=self.min_count,
                                     workers=self.workers)
      # Similarity of user_preferences
      similarity = model.wv.similarity(w1=self.user_preferences[0], w2=self.user_preferences[1])
      # Similarity of word in user_preferences
      similar_words = model.wv.most_similar(positive=[self.user_preferences[0], self.user_preferences[1]])
      self.user_preferences.append(similar_words[0][0])
    else:
      pass
    return user_preferences

## Recommender System

In [7]:
class RecommenderSystem:
  def __init__(self, data, city, metadata_tokenized):
    """
    Initialize RecommenderSystem class with required attributes.
    Parameters:
    - data (dict): Dictionary containing datasets.
    - city (str): The specific city's dataset to be used.
    - metadata_tokenized (list): List of tokenized metadata for the city.
    """
    self.data = data
    self.city = data[city]
    self.metadata_tokenized = metadata_tokenized
    self.vectorizer = None
    self.documents = None
    self.recommendation_result = None

  def fit(self):
    """
    Fit method to encode metadata with TF-IDF vectorization.
    """
    self.vectorizer = TfidfVectorizer()
    self.documents = self.vectorizer.fit_transform(self.metadata_tokenized)

  def recommend(self, user_preferences, top_recommend=15):
    """
    Recommend method to generate recommendations based on user preferences.
    Parameters:
    - user_preferences (list): List of user preferences.
    - top_recommend (int): Number of top recommendations to generate (default: 10).
    Returns:
    - list: A list of indices representing recommended items/documents.
    """
    self.list_appended = []

    for preference in user_preferences:
      preference_vector = self.vectorizer.transform([preference])
      distance = cosine_distances(preference_vector, self.documents)
      distance_sort = distance.argsort()[0, 0:top_recommend]
      self.list_appended.append(distance_sort)
    return self.list_appended

  def postprocessing(self):
    combined = list(OrderedDict.fromkeys(np.concatenate(self.list_appended)))
    self.recommendation_result = self.city.iloc[combined]
    return self.recommendation_result

## Filtering

In [8]:
class Filtering:
    def __init__(self, data):
        """
        Initialize Filtering class with data and recommendation attributes.

        Parameters:
        - data (DataFrame): Dataset containing information for filtering.
        """
        self.data = data
        self.recommendation_budget = None
        self.recommendation_distance = None

    def budgeting(self, budget):
        """
        Budgeting method to filter data based on budget constraints.

        Parameters:
        - budget (float): Maximum budget constraint.

        Returns:
        - DataFrame: Filtered data within the budget limit.
        """
        self.total_price = 0
        self.selected_rows = []

        for index, row in self.data.iterrows():
            if self.total_price + row['price'] <= budget:
                self.total_price += row['price']
                self.selected_rows.append(row)

        self.recommendation_budget = pd.DataFrame(self.selected_rows)
        return self.recommendation_budget, self.total_price

    def haversine_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate Haversine distance between two geographical coordinates.

        Parameters:
        - lat1 (float): Latitude of the first point.
        - lon1 (float): Longitude of the first point.
        - lat2 (float): Latitude of the second point.
        - lon2 (float): Longitude of the second point.

        Returns:
        - float: Haversine distance between the points.
        """
        R = 6371
        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon/2) * np.sin(dlon/2)
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        distance = R * c
        return distance

    def nearest_neighbor(self, points, start_lat, start_lon):
        """
        Find nearest neighbors using the Nearest Neighbor algorithm.

        Parameters:
        - points (array): Array of coordinates.
        - start_lat (float): Latitude of the starting point.
        - start_lon (float): Longitude of the starting point.

        Returns:
        - list: Route containing indices of nearest neighbor points.
        - float: Total distance of the route.
        """
        self.route = []
        self.total_distance = None
        start_point = np.array([[start_lat, start_lon]])
        points = np.concatenate((points, start_point), axis=0)

        n = len(points)
        visited = np.zeros(n, dtype=bool)
        visited[-1] = True
        route = [n - 1]
        total_dist = 0

        for i in range(n - 1):
            last_point = route[-1]
            min_dist = float('inf')
            nearest_point = None
            for j in range(n):
                if not visited[j] and j != last_point:
                    dist = self.haversine_distance(points[last_point][0], points[last_point][1], points[j][0], points[j][1])
                    if dist < min_dist:
                        min_dist = dist
                        nearest_point = j
            visited[nearest_point] = True
            route.append(nearest_point)
            total_dist += min_dist

        route.remove(n - 1)
        total_dist += self.haversine_distance(points[route[-1]][0], points[route[-1]][1], points[route[0]][0], points[route[0]][1])

        return route, total_dist

    def distance(self, recommendation_budget, start_lat, start_long):
        """
        Calculate distance and generate recommendations based on nearest neighbors.

        Parameters:
        - start_lat (float): Latitude of the starting point.
        - start_long (float): Longitude of the starting point.

        Returns:
        - list: Route containing indices of recommended points.
        - float: Total distance of the route.
        - DataFrame: Filtered data based on nearest neighbor route.
        """
        self.start_lat = start_lat
        self.start_long = start_long
        self.coordinates = recommendation_budget[['lat', 'long']].values

        route, total_distance = self.nearest_neighbor(self.coordinates, self.start_lat, self.start_long)
        self.route = route
        self.total_distance = total_distance
        self.recommendation_distance = recommendation_budget.iloc[self.route]
        return self.route, self.total_distance, self.recommendation_distance

# Thompson Sampling

In [9]:
class ThompsonSampling():
  def __init__(self):
    self.ratings = []
    self.exploitation_recommendations = []
    self.exploration_recommendations = []

  def bandit(self, ratings):
    sampled_indices = np.argmax(np.random.beta(1+ratings, 10))
    return sampled_indices

  def exploration(self, data, duration):
    for day in range(1, duration+1):
      exploitation_mask = data['rating'] >= 4.5
      self.exploitation_candidates = data[exploitation_mask]
      self.exploitation_candidates = self.exploitation_candidates[~self.exploitation_candidates['place_name'].isin(self.exploitation_recommendations + self.exploration_recommendations)]
      self.exploitation_recommendations.append(self.exploitation_candidates)
      self.exploration_candidates = data[~exploitation_mask]
      self.exploration_candidates = self.exploration_candidates[~self.exploration_candidates['place_name'].isin(self.exploitation_recommendations + self.exploration_recommendations)]
      self.exploration_recommendations.append(self.exploration_candidates)

      # Thompson sampling
      combine = pd.concat(self.exploitation_recommendations + self.exploration_recommendations, ignore_index=True)

      return combine

# Duration

In [10]:
class DurationPlaceDivider:
    def __init__(self, data):
        self.data = data
        self.slices = [data[i:i + 3][['id', 'place_name']].reset_index(drop=True) for i in range(0, len(data), 3)]

    def divide_durations_place_name(self, max_days=None):
        if max_days is None:
            max_days = len(self.slices)

        sliced_data = []
        for i, slices in enumerate(self.slices[:max_days], start=1):
            day_slices = []
            for _, row in slices.iterrows():
                day_slices.append({'id': row['id'], 'place_name': row['place_name']})
            sliced_data.append(day_slices)

        return sliced_data


# Carbon Footprint

In [11]:
class CarbonFootprint():
    def __init__(self):
        self.co2_emissions = None

    def predict(self, vehicle_type, distance):
        if vehicle_type == "car":
            return self.car(distance)
        elif vehicle_type == "bus":
            return self.bus(distance)
        elif vehicle_type == "motorbike":
            return self.motorbike(distance)

    def car(self, distance):
        model_car = tf.keras.models.load_model('/content/model_carbon_car.h5')
        input_data = np.array([distance])
        prediction = model_car.predict(input_data.reshape(1,1), verbose=0)
        return prediction[0][0].squeeze()

    def bus(self, distance):
        model_bus = tf.keras.models.load_model('/content/model_carbon_bus.h5')
        input_data = np.array([distance])
        prediction = model_bus.predict(input_data.reshape(1, 1), verbose=0)
        return prediction[0][0].squeeze()

    def motorbike(self, distance):
        model_motorbike = tf.keras.models.load_model('/content/model_carbon_motorbike.h5')
        input_data = np.array([distance])
        prediction = model_motorbike.predict(input_data.reshape(1, 1), verbose=0)
        return prediction[0][0].squeeze()

    def predict_all(self, distance):
        self.distance = distance
        results = []
        vehicle_type = ['car', 'bus', 'motorbike']

        for vehicle in vehicle_type:
            result = self.predict(vehicle, self.distance)
            results.append(f"{result:.2f} kg CO2")

        return results

# City

In [12]:
class City():
  def __init__(self):
    self.city = None

  def carbon(self, carbon):
    self.co2_car = carbon[0]
    self.co2_bus = carbon[1]
    self.co2_motorbike = carbon[2]

  def select_city(self, city, task):
    if city == "jakarta":
        if task == 1:
            return self.jakarta("coordinates")
        elif task == 2:
            return self.jakarta("price_emissions")
    elif city == "yogyakarta":
        if task == 1:
            return self.yogyakarta("coordinates")
        elif task == 2:
            return self.yogyakarta("price_emissions")
    elif city == "bandung":
        if task == 1:
            return self.bandung("coordinates")
        elif task == 2:
            return self.bandung("price_emissions")
    elif city == "semarang":
        if task == 1:
            return self.semarang("coordinates")
        elif task == 2:
            return self.semarang("price_emissions")
    elif city == "surabaya":
        if task == 1:
            return self.surabaya("coordinates")
        elif task == 2:
            return self.surabaya("price_emissions")
    elif city == "bengkulu":
        if task == 1:
            return self.bengkulu("coordinates")
        elif task == 2:
            return self.bengkulu("price_emissions")
    elif city == "banjarbaru":
        if task == 1:
            return self.banjarbaru("coordinates")
        elif task == 2:
            return self.banjarbaru("price_emissions")
    elif city == "denpasar":
        if task == 1:
            return self.denpasar("coordinates")
        elif task == 2:
            return self.denpasar("price_emissions")
    elif city == "maluku":
        if task == 1:
            return self.maluku("coordinates")
        elif task == 2:
            return self.maluku("price_emissions")
    elif city == "jayapura":
        if task == 1:
            return self.jayapura("coordinates")
        elif task == 2:
            return self.jayapura("price_emissions")

  def jakarta(self, condition):
    if condition == "coordinates":
      start_latitude = -6.2651234
      start_longitude = 106.8678393
      return start_latitude, start_longitude
    elif condition == "price_emissions":
      car = 20000
      bus = 3500
      motor = 12000
      return "jakarta", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

  def yogyakarta(self, condition):
    if condition == "coordinates":
      start_latitude = -7.7889482
      start_longitude = 110.4281026
      return start_latitude, start_longitude
    elif condition == "price_emissions":
      car = 20000
      bus = 3500
      motor = 12000
      return "yogyakarta", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

  def bandung(self, condition):
    if condition == "coordinates":
      start_latitude = -6.9146413
      start_longitude = 107.5998627
      return start_latitude, start_longitude
    elif condition == "price_emissions":
      car = 20000
      bus = 5000
      motor = 12000
      return "bandung", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

  def semarang(self, condition):
    if condition == "coordinates":
      start_latitude = -6.9644063
      start_longitude = 110.4253333
      return start_latitude, start_longitude
    elif condition == "price_emissions":
      car = 20000
      bus = 3500
      motor = 12000
      return "semarang", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

  def surabaya(self, condition):
    if condition == "coordinates":
      start_latitude = -7.3719669
      start_longitude = 112.7756714
      return start_latitude, start_longitude
    elif condition == "price_emissions":
      car = 20000
      bus = 5000
      motor = 12000
      return "surabaya", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

  def bengkulu(self, condition):
    if condition == "coordinates":
      start_latitude = -3.8606521
      start_longitude = 102.3368196
      return start_latitude, start_longitude
    elif condition == "price_emissions":
      car = 20000
      bus = 5000
      motor = 12000
      return "bengkulu", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

  def banjarbaru(self, condition):
    if condition == "coordinates":
      start_latitude = -3.4360195
      start_longitude = 114.7582452
      return start_latitude, start_longitude
    elif condition == "price_emissions":
      car = 20000
      bus = 5000
      motor = 12000
      return "banjarbaru", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

  def denpasar(self, condition):
    if condition == "coordinates":
      start_latitude = -8.7469693
      start_longitude = 115.1642308
      return start_latitude, start_longitude
    elif condition == "price_emissions":
      car = 20000
      bus = 5000
      motor = 12000
      return "denpasar", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

  def maluku(self, condition):
    if condition == "coordinates":
      start_latitude = -3.7048911
      start_longitude = 128.0862563
      return start_latitude, start_longitude
    elif condition ==  "price_emissions":
      car = 20000
      bus = 3500
      motor = 10000
      return "maluku", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

  def jayapura(self, condition):
    if condition == "coordinates":
      start_latitude = -2.5772586
      start_longitude = 140.51555
      return start_latitude, start_longitude
    elif condition == "price_emissions":
      car = 50000
      bus = 50000
      motor = 15000
      return "jayapura", car, self.co2_car, bus, self.co2_bus, motor, self.co2_motorbike

# Pipeline

In [17]:
def make_pipeline(datasets, city, budget, duration, user_preferences_1, user_preferences_2=None):
  # Get user city
  user_city = City()
  start_latitude, start_longitude = user_city.select_city(city, 1)
  # Preprocessing
  preprocess = TextPreprocessing(datasets, city)
  stemmed_documents = preprocess.stemming()
  datasets, tokenized_strings = preprocess.tokenizer(stemmed_documents)
  # User Input
  user = User(datasets)
  combined, user_preferences, duration, budget = user.input(city, budget, duration, user_preferences_1, user_preferences_2)
  # Preferences
  preferences = GenerateNewPreference()
  user_preferences_new = preferences.fit(combined, user_preferences)
  # Recommender System
  recommender = RecommenderSystem(datasets, city, tokenized_strings)
  recommender.fit()
  recommender.recommend(user_preferences)
  recommendation_result = recommender.postprocessing()
  # Thompson Sampling
  sample = ThompsonSampling()
  recommendation_exploration = sample.exploration(recommendation_result, duration)
  # Filtering
  filter = Filtering(recommendation_exploration)
  recommendation_budget, total_budget = filter.budgeting(budget)
  recommendation_budget = recommendation_budget.sort_values('price', ascending=False)
  # Duration
  recommendation_duration = recommendation_budget[:(duration*3)]
  # Distance
  route, total_distance, recommendation_distance = filter.distance(recommendation_duration, start_latitude, start_longitude)
  # Divide by duration
  duration_divide = DurationPlaceDivider(recommendation_distance)
  day_recommendation = duration_divide.divide_durations_place_name(duration)
  # Calculate Carbon Footprint
  footprint = CarbonFootprint()
  results = footprint.predict_all(total_distance)
  # Price and CO2 Emissions
  city_price_emissions = City()
  city_price_emissions.carbon(results)
  city_price_emissions_data = city_price_emissions.select_city(city, 2)
  return recommendation_distance

In [18]:
recommendation = make_pipeline(datasets, "bandung", 9000, 3, "wisata", "alam")
recommendation

,id,place_name,description,category,city,province,price,rating,coordinate,lat,long,metadata,dir,metadata_tokenized
10,26,Taman Balai Kota Bandung,Taman Balai Kota Bandung merupakan sebuah tama...,Tempat Rekreasi,Bandung,Jawa Barat,0,4.6,"{'lat': -6.912966000000001, 'lng': 107.6096031}",-6.912966,107.609603,taman balai kota bandung tempat rekreasi taman...,images/bandung/image_26,taman balai kota bandung rekreasi taman balai ...
8,75,Taman Sejarah Bandung,Taman Sejarah Bandung adalah taman yang terlet...,Tempat Rekreasi,Bandung,Jawa Barat,0,4.6,"{'lat': -6.9103086, 'lng': 107.6098619}",-6.910309,107.609862,taman sejarah bandung tempat rekreasi taman se...,images/bandung/image_75,taman sejarah bandung rekreasi taman sejarah b...
9,17,Alun-Alun Kota Bandung,Alun-alun Bandung adalah pusat kota Bandung ya...,Tempat Rekreasi,Bandung,Jawa Barat,0,4.6,"{'lat': -6.9218571, 'lng': 107.6070141}",-6.921857,107.607014,alun-alun kota bandung tempat rekreasi alun-al...,images/bandung/image_17,alun kota bandung rekreasi alun bandung pusat ...
36,47,Monumen Bandung Lautan Api,"Monumen Bandung Lautan Api, merupakan monumen ...",Sejarah dan Budaya,Bandung,Jawa Barat,0,4.3,"{'lat': -6.9338497, 'lng': 107.604925}",-6.933850,107.604925,monumen bandung lautan api sejarah dan budaya ...,images/bandung/image_47,monumen bandung laut api sejarah budaya monume...
20,104,Tafso Barn,Nama Punclut mungkin sudah cukup akrab di teli...,Tempat Rekreasi,Bandung,Jawa Barat,0,4.2,"{'lat': -6.8426446, 'lng': 107.6228408}",-6.842645,107.622841,tafso barn tempat rekreasi nama punclut mungki...,images/bandung/image_104,tafso barn rekreasi nama punclut akrab telinga...
17,56,Caringin Tilu,"Bandung tidak pernah kehilangan pesonanya, wis...",Alam,Bandung,Jawa Barat,0,4.4,"{'lat': -6.858046399999999, 'lng': 107.6647084}",-6.858046,107.664708,caringin tilu alam bandung tidak pernah kehila...,images/bandung/image_56,caringin tilu alam bandung hilang pesona wisat...
6,28,Gunung Manglayang,Gunung Manglayang adalah sebuah gunung bertipe...,Pegunungan,Bandung,Jawa Barat,7500,4.5,"{'lat': -6.876111099999999, 'lng': 107.7436111}",-6.876111,107.743611,gunung manglayang pegunungan gunung manglayang...,images/bandung/image_28,gunung manglayang gunung gunung manglayang bua...
30,100,Taman Bunga Cihideung,Taman Bunga Cihideung mempunyai pesona alam ya...,Tempat Rekreasi,Bandung,Jawa Barat,0,4.1,"{'lat': -6.8038883, 'lng': 107.5866313}",-6.803888,107.586631,taman bunga cihideung tempat rekreasi taman bu...,images/bandung/image_100,taman bunga cihideung rekreasi taman bunga cih...
4,113,Kebun Tanaman Obat Sari Alam,Kebun Tanaman Obat Sari Alam lebih terkenal de...,Tempat Rekreasi,Bandung,Jawa Barat,0,4.9,"{'lat': -7.07723161619254, 'lng': 107.48667876...",-7.077232,107.486679,kebun tanaman obat sari alam tempat rekreasi k...,images/bandung/image_113,kebun tanam obat sari alam rekreasi kebun tana...
